<a href="https://colab.research.google.com/github/LLF00/datascience1/blob/main/recommendation_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163474 sha256=edd679758b8314ab28635b4b426b783065c73590dfa7972e2499a9ca756ac6be
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate

In [ ]:
ratings = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/ratings.csv')
movies = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/movies.csv')

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [ ]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html


In [ ]:
data = pd.merge(ratings,movies, on='movieId')

In [ ]:
data.describe()

,userId,movieId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [ ]:
reader = Reader(rating_scale=(0.5,5))

In [ ]:
data = Dataset.load_from_df(data[['userId', 'movieId','rating']], reader)

In [ ]:
data

In [ ]:
params = {'name':'cosine', 'user_based':False}

In [ ]:
model = KNNBasic(sim_options=params)

In [ ]:
cross_validate(model, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9809  0.9725  0.9783  0.9676  0.9722  0.9743  0.0047  
MAE (testset)     0.7619  0.7533  0.7588  0.7548  0.7565  0.7571  0.0031  
Fit time          9.39    8.83    8.57    8.38    8.93    8.82    0.34    
Test time         10.59   8.96    10.89   10.19   10.47   10.22   0.67    


{'test_rmse': array([0.98094017, 0.97252739, 0.97826829, 0.96762147, 0.97215999]),
 'test_mae': array([0.76194404, 0.75327343, 0.75883184, 0.75480828, 0.75653468]),
 'fit_time': (9.38684606552124,
  8.834189891815186,
  8.572052478790283,
  8.379212617874146,
  8.932380199432373),
 'test_time': (10.587883949279785,
  8.960402011871338,
  10.89359736442566,
  10.189201831817627,
  10.469653129577637)}

In [ ]:
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2)

In [ ]:
model2 = KNNBasic(sim_options={'user_based':True})
model2.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9632


0.9632082247605803

In [ ]:
model3 = KNNBasic(min_k = 5, k=10, sim_options={'name':'cosine','user_based':True})
model3.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9632


0.9632082247605803

In [ ]:
model4 = KNNBasic(min_k=5, k=100, sim_options={'name':'pearson', 'user_based':True})
model4.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9632


0.9632082247605803

In [ ]:
model5 = KNNBasic(min_k=5, k=100, sim_options={'name':'msd', 'user_based':True})
model5.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9632


0.9632082247605803

In [ ]:
model2.predict(668,16)

Prediction(uid=668, iid=16, r_ui=None, est=3.6060679154393127, details={'actual_k': 40, 'was_impossible': False})

In [ ]:
movies[movies['movieId']==16]

,movieId,title,genres
15,16,Casino (1995),Crime|Drama


In [ ]:
final_dataset = pd.merge(movies,ratings, on='movieId')

In [ ]:
final_dataset['predicted_rating'] = final_dataset.apply(lambda x:model2.predict(x['userId'],x['movieId']).est, axis=1)

In [ ]:
final_dataset.head()

,movieId,title,genres,userId,rating,timestamp,predicted_rating,svd_pred
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895,4.630253,4.209186
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039,4.097617,3.471735
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933,4.425728,4.207704
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810,3.899577,3.904258
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286,4.051380,4.000024


In [ ]:
#SVD algo

from surprise import SVD

In [ ]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8683  0.8609  0.8795  0.8666  0.8694  0.8690  0.0060  
Fit time          1.63    1.58    1.59    1.59    2.36    1.75    0.30    
Test time         0.12    0.14    0.13    0.39    0.21    0.20    0.10    


{'test_rmse': array([0.86830507, 0.86090437, 0.87952305, 0.86662587, 0.86941922]),
 'fit_time': (1.6326265335083008,
  1.5805103778839111,
  1.593092679977417,
  1.5877811908721924,
  2.3554847240448),
 'test_time': (0.12460184097290039,
  0.14155244827270508,
  0.12641358375549316,
  0.39497900009155273,
  0.20667052268981934)}

In [ ]:
trainset = data.build_full_trainset()

In [ ]:
svd.fit(trainset)

In [ ]:
final_dataset['svd_pred'] = final_dataset.apply(lambda x: svd.predict(x['userId'],x['movieId']).est, axis=1)

In [ ]:
final_dataset.head()

,movieId,title,genres,userId,rating,timestamp,predicted_rating,svd_pred
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895,4.630253,4.209186
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039,4.097617,3.471735
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933,4.425728,4.207704
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810,3.899577,3.904258
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286,4.051380,4.000024


In [ ]:
movies[movies['movieId']==10]

,movieId,title,genres
9,10,GoldenEye (1995),Action|Adventure|Thriller
